<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/feature%2Fadd-data-generators-to-fix-ram-problem/prepare_data/combine_npz_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Combine npz files

This is a notebook for the last step in the prepare data pipeline as we did not have enough RAM to run it locally. To train the model on the entire dataset it is more convenient to have all tile-arrays of all images in one .npz file. 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#! ls
%cd drive/MyDrive/MachineLearning
%cd Geospatial_ML
! ls

/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
architecture.drawio  colab.py	       experiments   __pycache__
colab-new.py	     data_exploration  models	     README.md
colab_new.py	     evaluation        prepare_data  requirements.txt


In [2]:
import numpy as np
import os
import pickle

In [5]:
# Problem: running out of ram when trying to save more than 5 images in one compressed npz, crashing always just at the savez_compressed() step
# => combining only 5 images into one file and then trying to combine those 2 files if possible
# => better way? Why does savez_compressed() consume most RAM?
def combine_npz_arrays(data_path):
    count = 0
    arrays_dict = {}
    for file in os.listdir(data_path):
        if file != '2022_08_09.npz' and count < 5: # Todo: find out problem with image 2022_08_09 => crashes when trying to access x_input
            print(f'Adding image {file}')
            array = np.load(f'{data_path}/{file}')
            x_input = array['x_input']
            y_mask = array['y_mask']
            if len(arrays_dict) < 1:
                arrays_dict['x_input'] = x_input
                arrays_dict['y_mask'] = y_mask
            else:
                arrays_dict['x_input'] = np.concatenate((arrays_dict['x_input'], x_input), axis=0)
                arrays_dict['y_mask'] = np.concatenate((arrays_dict['y_mask'], y_mask), axis=0)
        count += 1
    # test time to execute and file size of np.savez and np.savez_compressed
    np.savez(f'{data_path}/all_images_1', **arrays_dict)
    print('Combined all compressed numpy images into one single file.')

combine_npz_arrays("../data_colab/256_200")

Adding image 2022_10_13.npz
Adding image 2022_07_15.npz
Adding image 2022_09_18.npz
Adding image 2022_06_20.npz
Adding image 2022_10_23.npz
Combined all compressed numpy images into one single file.


In [4]:
def combine_npz_arrays(data_path):
    count = 0
    arrays_dict = {}
    for file in os.listdir(data_path):
        if file != '2022_08_09.npz' and count > 4 and count < 10: # Todo: find out problem with image 2022_08_09 => crashes when trying to access x_input
            print(f'Adding image {file}')
            array = np.load(f'{data_path}/{file}')
            x_input = array['x_input']
            y_mask = array['y_mask']
            if len(arrays_dict) < 1:
                arrays_dict['x_input'] = x_input
                arrays_dict['y_mask'] = y_mask
            else:
                arrays_dict['x_input'] = np.concatenate((arrays_dict['x_input'], x_input), axis=0)
                arrays_dict['y_mask'] = np.concatenate((arrays_dict['y_mask'], y_mask), axis=0)
        count += 1
    np.savez_compressed(f'{data_path}/all_images_2', **arrays_dict)
    print('Combined all compressed numpy images into one single file.')

combine_npz_arrays("../data_colab/256_200")

Adding image 2022_07_25.npz
Adding image 2022_08_04.npz
Adding image 2022_07_10.npz
Adding image 2022_07_30.npz
Adding image 2022_08_14.npz
Combined all compressed numpy images into one single file.


In [ ]:
data_path = '../data_colab/256_200'
combined_1 = np.load(f'{data_path}/all_images_1.npz')
combined_2 = np.load(f'{data_path}/all_images_2.npz')
print('loaded combined 1')
arrays_dict = {}
arrays_dict['x_input'] = combined_1['x_input']
arrays_dict['y_mask'] = combined_1['y_mask']
print('loaded combined 2')
arrays_dict['x_input'] = np.concatenate((arrays_dict['x_input'], combined_2['x_input']), axis=0)
arrays_dict['y_mask'] = np.concatenate((arrays_dict['y_mask'], combined_2['y_mask']), axis=0)
print('concatenated arrays')
# test time to execute and file size of np.savez and np.savez_compressed
np.savez(f'{data_path}/all_images_1_2', **arrays_dict)
print('finish saving')

loaded combined 1
loaded combined 2
